In [2]:
import wandb
from wandb.keras import WandbCallback


import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, MaxPool2D, TimeDistributed, LSTM,Conv1D,MaxPool1D,Bidirectional
from tensorflow.keras.models import Model

from TranscriptionDataset.TensorflowMusicMetaSequence import MusicMetaSequence
from TUtils import get_timestamp

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.12.0


In [3]:
logdir = f"TBLogs/{get_timestamp()}"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
wandb.tensorboard.patch(root_logdir=logdir)
wandb.init(project='TFArrangement')
print(f"Tensorboard : {logdir}")

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


Tensorboard : TBLogs/1687274624


In [4]:
trainSequence = MusicMetaSequence("Trainsets/S_Tier_Mel.hdf5")
testSequence = MusicMetaSequence("Trainsets/Mini_Validation_Test.hdf5",shuffle=False)
# trainSequence[0][0].shape,trainSequence[0][1].shape

In [5]:
input_layer = Input(shape=(None,256),batch_size=1)
x = (Conv1D(data_format="channels_last",filters=128, kernel_size=5, padding="same", activation="relu"))(input_layer)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=64, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = (Conv1D(data_format="channels_last",filters=8, kernel_size=5, padding="same", activation="relu"))(x)
x = (MaxPool1D(pool_size=2))(x)
x = Bidirectional(LSTM(128))(x)
x = Dense(128,activation="relu")(x)
# output = Dense(1,activation="sigmoid")(x)
outputs = []
for i in range(6):
    outputs.append(Dense(1,name=f'output_{i}',activation="sigmoid")(x))

model = Model(inputs=input_layer, outputs=outputs)

print(model.summary())

2023-06-20 20:53:57.803441: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:53:57.809118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:53:57.809177: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:53:57.812603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-20 20:53:57.812681: I tensorflow/compile

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(1, None, 256)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (1, None, 128)       163968      ['input_1[0][0]']                
                                                                                                  
 max_pooling1d (MaxPooling1D)   (1, None, 128)       0           ['conv1d[0][0]']                 
                                                                                                  
 conv1d_1 (Conv1D)              (1, None, 64)        41024       ['max_pooling1d[0][0]']          
                                                                                              

In [6]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.001)
loss = {}
metrics = {}
for i in range(6):
    loss[f'output_{i}'] = tf.keras.losses.BinaryFocalCrossentropy()
    metrics[f"output_{i}"] = tf.keras.metrics.BinaryAccuracy()
model.compile(optimizer=optimizer,loss=loss,metrics=metrics)

In [8]:
model.fit(trainSequence,epochs=10,verbose=1,callbacks=[tensorboard_callback])

Epoch 1/10


2023-06-20 21:02:30.419538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1269/1269 [==============================] - 188s 148ms/step - loss: 0.2164 - output_0_loss: 0.0179 - output_1_loss: 0.0953 - output_2_loss: 0.0028 - output_3_loss: 0.0454 - output_4_loss: 0.0526 - output_5_loss: 0.0024 - output_0_binary_accuracy: 0.9898 - output_1_binary_accuracy: 0.8920 - output_2_binary_accuracy: 0.9992 - output_3_binary_accuracy: 0.9606 - output_4_binary_accuracy: 0.9614 - output_5_binary_accuracy: 0.9992
Epoch 2/10
1269/1269 [==============================] - 182s 143ms/step - loss: 0.2037 - output_0_loss: 0.0206 - output_1_loss: 0.0936 - output_2_loss: 0.0027 - output_3_loss: 0.0342 - output_4_loss: 0.0500 - output_5_loss: 0.0026 - output_0_binary_accuracy: 0.9898 - output_1_binary_accuracy: 0.8920 - output_2_binary_accuracy: 0.9992 - output_3_binary_accuracy: 0.9771 - output_4_binary_accuracy: 0.9614 - output_5_binary_accuracy: 0.9992
Epoch 3/10
 540/1269 [===========>..................] - ETA: 1:44 - loss: 0.1954 - output_0_loss: 0.0176 - output_1_loss: 0.0917 

KeyboardInterrupt: 

In [8]:
preds = model.predict(testSequence,verbose=1)
expected_test = testSequence.get_expected_output()

2023-06-20 20:20:42.378639: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


27/27 [==============================] - 3s 104ms/step


In [9]:
expected_test

array([[1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 1., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.],
       [1., 0., 0., 1., 0., 0.]], dtype=float32)

In [10]:
score = model.evaluate(testSequence, verbose = 1)

2023-06-20 21:10:29.679691: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-20 21:10:30.248309: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-20 21:10:30.250314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_gra

27/27 [==============================] - 6s 161ms/step - loss: 0.0841 - output_0_loss: 0.0011 - output_1_loss: 0.0689 - output_2_loss: 2.6460e-06 - output_3_loss: 0.0051 - output_4_loss: 0.0090 - output_5_loss: 1.8295e-05 - output_0_binary_accuracy: 1.0000 - output_1_binary_accuracy: 0.9259 - output_2_binary_accuracy: 1.0000 - output_3_binary_accuracy: 1.0000 - output_4_binary_accuracy: 1.0000 - output_5_binary_accuracy: 1.0000


In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.14047811925411224
Test accuracy: 0.0009182813810184598


In [17]:
model_name = "ArrangementModel_S_Tier_Only"
model.save(f"Models/{model_name}_{get_timestamp()}.h5")

In [9]:
wandb.finish()